In [ ]:
!pip install -q xlrd
!pip install unidecode
!pip install python-Levenshtein
!pip install fuzzywuzzy
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

POI_df = pd.read_excel("/content/drive/My Drive/Workspace/Balad Data Analyst - 2nd Task/POI info.xlsx")
categ_df = pd.read_excel("/content/drive/My Drive/Workspace/Balad Data Analyst - 2nd Task/category.xlsx")

     |████████████████████████████████| 241 kB 33.7 MB/s 
     |████████████████████████████████| 50 kB 6.4 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149861 sha256=c97643355d09ed782b5627cb1cc87fb51546d0f8a15328b7b357fa67fa447b94
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein
Mounted at /content/drive


###Preprocessing

In [ ]:
# General preprocessing
# cleaning unicodes from text columns
def clean_text(col):
  # return the list of decoded cell in the Series instead '
  for c in col:
    if c is not None:
      text_encode = str(c).encode(encoding="ascii", errors="ignore") # encoding the text to ASCII format
      text_decode = text_encode.decode("utf-8")
      str(c).replace('\u200c', ' ')
      cleaned_text = " ".join([word for word in text_decode.split()]) # remove extra whitespace 
      c = cleaned_text
  return col

# droping rows containing NaN from categories df
categ_df = categ_df.dropna(subset = ['دسته بندی'])
categ_df = categ_df.reset_index(drop=True)

categ_df['دسته بندی'] = clean_text(categ_df['دسته بندی'])
POI_df['name'] = clean_text(POI_df['name'])
POI_df['city'] = clean_text(POI_df['city'])
POI_df['category'] = clean_text(POI_df['category'])
POI_df['address'] = clean_text(POI_df['address'])
del POI_df['Unnamed: 0']

print("POIs:\n", POI_df)
print("Categories:", categ_df)

POIs:
               Id  ... review
0     1838574004  ...      3
1     5007356428  ...      4
2     5015543907  ...      3
3     5009811456  ...      4
4     5007351985  ...      3
...          ...  ...    ...
9970  5067576349  ...     44
9971  5074167388  ...      0
9972         NaN  ...    NaN
9973         NaN  ...    NaN
9974              ...    NaN

[9975 rows x 8 columns]
Categories:                  دسته بندی Unnamed: 1  ... Unnamed: 4  Unnamed: 5
0                بیمارستان        NaN  ...        NaN         NaN
1            کلینیک درمانی        NaN  ...        NaN         NaN
2                 درمانگاه        NaN  ...        NaN         NaN
3                   بهداری        NaN  ...        NaN         NaN
4           ساختمان پزشکان        NaN  ...        NaN         NaN
..                     ...        ...  ...        ...         ...
376                    دشت        NaN  ...        NaN         NaN
377               باغ میوه        NaN  ...        NaN         NaN
378           


###Task 1 : Finding Similar POIs

In [ ]:
# finding similar between two cols/rows

def commonWords (phrase1, phrase2):

    list1 = str(phrase1).split()
    list2 = str(phrase2).split()
    list3 = set(list1) & set(list2)
    list4 = sorted(list3, key = lambda k : list1.index(k))
    try:
      list4.remove("و")
    except ValueError:
      pass
    return list4


def similarities (row1, row2):

  listoflists = [] 
  for val1 in row1:
    diff = []
    for val2 in row2:
      #if common is not None and len(common) > 1: # not null and long enough
      diff.append(commonWords(str(val1), str(val2)))
    listoflists.append(diff)
  CW_df = pd.DataFrame(listoflists)
  return CW_df

def get_len_string (strg):

  wordlist = list(set(str(strg).split()))
  try:
    wordlist.remove("و")
  except ValueError:
    pass
  return len(wordlist)


In [ ]:
# similarity ratio among NAMES

name_similarity = similarities(POI_df.name, POI_df.name)
names_sim_df = name_similarity.applymap(len) # applies function len to all cells of df

namelen_df = pd.DataFrame(POI_df.name) # ratio dominator
namelen_df = namelen_df.applymap(get_len_string)

names_sim_df = names_sim_df.div(namelen_df.values)
print("names similaarity ratio:\n", names_sim_df)
name_similarity = names_sim_df.where(names_sim_df.values != np.diag(names_sim_df),0,names_sim_df.where(names_sim_df.values != np.flipud(names_sim_df).diagonal(0),0,inplace=True)) # sets diagonal elements to zero


names similaarity ratio:
       0     1     2     3     4     5     ...  9969  9970  9971  9972  9973  9974
0      1.0   0.0   0.0   0.0  0.00   0.0  ...   0.0  0.00   0.0   0.0   0.0   0.0
1      0.0   1.0   0.0   0.0  0.00   0.0  ...   0.0  0.00   0.0   0.0   0.0   0.0
2      0.0   0.0   1.0   0.0  0.00   0.0  ...   0.0  0.00   0.0   0.0   0.0   0.0
3      0.0   0.0   0.0   1.0  0.00   0.0  ...   0.0  0.00   0.0   0.0   0.0   0.0
4      0.0   0.0   0.0   0.0  1.00   0.0  ...   0.0  0.25   0.0   0.0   0.0   0.0
...    ...   ...   ...   ...   ...   ...  ...   ...   ...   ...   ...   ...   ...
9970   0.0   0.0   0.0   0.0  0.25   0.0  ...   0.0  1.00   0.0   0.0   0.0   0.0
9971   0.0   0.0   0.0   0.0  0.00   0.0  ...   0.0  0.00   1.0   0.0   0.0   0.0
9972   0.0   0.0   0.0   0.0  0.00   0.0  ...   0.0  0.00   0.0   1.0   1.0   1.0
9973   0.0   0.0   0.0   0.0  0.00   0.0  ...   0.0  0.00   0.0   1.0   1.0   1.0
9974   0.0   0.0   0.0   0.0  0.00   0.0  ...   0.0  0.00   0.0   1.0   

In [ ]:
# similarity ratio among PHONES

# preprocess on phones
# turning arabic/persian numbers to english
from unidecode import unidecode
import re

# checking if there is an address writen in the phones field
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

print("POIs:\n", POI_df.phones)
for numIndex, numbers in POI_df['phones'].iteritems():
  if not isEnglish(str(numbers)):
    decodedNum = unidecode(numbers)
    if str(decodedNum).find('+98') != -1: # replace +98 with 0
      print("here:", numbers)
      decodedNum = re.sub(r"\+98", "0", str(decodedNum))

    # seeing if there are non-digits in the string
    hasDigits = re.findall('\+?[0-9]+', decodedNum)

    # if there is a non-digit writen in the phones field,
    ## puts it in the corresponding address field if it is null
    if len(hasDigits) == 0:
      POI_df.at[numIndex, 'address'] = str(numbers) + str(POI_df.address[numIndex])
      print("address field filled")
      if POI_df['location'][numIndex].find("POINT") == -1 and POI_df['location'][numIndex]==POI_df['location'][numIndex]: # there is number in location field
        POI_df.at[numIndex, 'phones'] = POI_df['location'][numIndex]
        if POI_df['review'][numIndex].find("POINT") != -1: # finds substring POINT in review section
          POI_df.at[numIndex, 'location'] = POI_df['review'][numIndex]
          POI_df.at[numIndex, 'review'] = 0
      else:
        POI_df.at[numIndex, 'phones'] = re.findall('\+?[0-9]+', str(numbers))     
    else:
      POI_df.at[numIndex, 'phones'] = re.findall('\+?[0-9]+', str(decodedNum))
      #print("REPLACED VALUE:", POI_df.phones[numIndex])
  else:
    if str(numbers).find('+98') != -1: # replace +98 with 0 \+?\d+
      numbers = re.sub(r'\+98', '0', str(numbers))
      #print("after:", numbers)
    POI_df.at[numIndex, 'phones'] = re.findall('\+?[0-9]+', str(numbers))

POIs:
 0                               []
1       [02166126561, 02166129785]
2                               []
3                               []
4                               []
                   ...            
9970                 [02166702194]
9971                 [03136513659]
9972                           NaN
9973                           NaN
9974                           NaN
Name: phones, Length: 9975, dtype: object
address field filled
address field filled
address field filled
address field filled
address field filled


In [ ]:
def convert_phone(phones):
  indexes = [] 
  results = []
  for i in range(len(phones)):
    item = phones[i]
    if type(item) == str and item != '[]' : # check type and emptyness
      numbers = item.replace('[' , ' ').replace(']' , ' ').replace(',' , ' ').split()
      for number in numbers:
        results.append(number)
        indexes.append(i) 

    elif type(item) == list and len(item) > 0:
      for number in item:
        results.append(str(number)) 
        indexes.append(i)
    else:
      #print('error: type ' ,type(item)) # empty lists
      pass

  return indexes , results # results len is 8288


idx , new_phones = convert_phone(POI_df['phones'])

In [ ]:
def compare_phones(phones):

    phones = np.array(phones)

    temp_phone = phones[:, None]

    results = temp_phone == temp_phone.T

    return results

new_phones_similarity = compare_phones(new_phones)
print(new_phones_similarity)

[[ True False False ... False False False]
 [False  True False ... False False False]
 [False False  True ... False False False]
 ...
 [False False False ...  True False False]
 [False False False ... False  True False]
 [False False False ... False False  True]]


In [ ]:
def decode_phone(indexes , n , similarity):

    m = len(indexes)
    results = np.zeros((n,n))

    for i in range(0,m):
        for j in range(0,m):
            if similarity[i][j] :
                row_id = indexes[i]
                col_id = indexes[j]
                if row_id != col_id: 
                    results[row_id][col_id] = 1

    return pd.DataFrame(results)


phone_similarity = decode_phone(idx , len(POI_df['phones']) , new_phones_similarity)
print("Total number of exact matches among phones is:", (phone_similarity == 1).astype(int).sum().sum()/2)
print("\n", phone_similarity)

Total number of exact matches among phones is: 872.0

       0     1     2     3     4     5     ...  9969  9970  9971  9972  9973  9974
0      0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
1      0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
2      0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
3      0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
4      0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
...    ...   ...   ...   ...   ...   ...  ...   ...   ...   ...   ...   ...   ...
9970   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
9971   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
9972   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
9973   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
9974   0.0   0.0   0.0   0.0   0.0   0.0  .

In [ ]:
# similarity ratio among CITY
def convert_phrase(phrases):
  indexes = [] 
  results = []
  for i in range(len(phrases)):
    item = phrases[i]
    if type(item) == str and item != '[]' : # check type and emptyness
      try:
        item = item.replace('طهران' , 'تهران')
      except:
        pass
      results.append(item)
      indexes.append(i)
    else:
      #print('error: type ' ,type(item)) # empty lists
      pass
  return indexes , results


idx , new_cities = convert_phrase(POI_df['city'])
new_cities_similarity = compare_phones(new_cities)
print(new_cities_similarity)
city_similarity = decode_phone(idx , len(POI_df['city']) , new_cities_similarity)
print("\n", city_similarity)

[[ True False False ... False False False]
 [False  True  True ...  True  True False]
 [False  True  True ...  True  True False]
 ...
 [False  True  True ...  True  True False]
 [False  True  True ...  True  True False]
 [False False False ... False False  True]]

       0     1     2     3     4     5     ...  9969  9970  9971  9972  9973  9974
0      0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
1      0.0   0.0   0.0   1.0   1.0   1.0  ...   1.0   1.0   0.0   0.0   0.0   0.0
2      0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
3      0.0   1.0   0.0   0.0   1.0   1.0  ...   1.0   1.0   0.0   0.0   0.0   0.0
4      0.0   1.0   0.0   1.0   0.0   1.0  ...   1.0   1.0   0.0   0.0   0.0   0.0
...    ...   ...   ...   ...   ...   ...  ...   ...   ...   ...   ...   ...   ...
9970   0.0   1.0   0.0   1.0   1.0   1.0  ...   1.0   0.0   0.0   0.0   0.0   0.0
9971   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   

In [ ]:
# similarity ratio among CATEGORY

idx , new_categories = convert_phrase(POI_df['category'])
new_categories_similarity = compare_phones(new_categories)
category_similarity = decode_phone(idx , len(POI_df['category']) , new_categories_similarity)
print("\n", category_similarity)


       0     1     2     3     4     5     ...  9969  9970  9971  9972  9973  9974
0      0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
1      0.0   0.0   0.0   1.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
2      0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
3      0.0   1.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
4      0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
...    ...   ...   ...   ...   ...   ...  ...   ...   ...   ...   ...   ...   ...
9970   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
9971   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
9972   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
9973   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
9974   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0

[9975 rows x 

In [ ]:
# similarity ratio among LOCATIONS
def convert_lat_long(locations):
    lats = []
    lons = []
    for i in range(0,len(locations)):
        item = locations[i]
        if type(item) in [float,int] :
            lats.append(np.nan)
            lons.append(np.nan)
        else:
            item = item.replace('POINT(' , ' ').replace(')' , ' ').split()
            try:
                lats.append(float(item[0]))
                lons.append(float(item[1]))
            except:
                lats.append(np.nan)
                lons.append(np.nan)
    return pd.DataFrame({'lat': lats, 'long': lons})    

lat_long_df = convert_lat_long(POI_df['location'])
print(lat_long_df)

            lat       long
0     50.919439  35.758466
1     51.386519  35.707696
2     48.907061  33.964059
3     51.411348  35.757394
4     51.402944  35.697201
...         ...        ...
9970  51.424622  35.701474
9971  51.679108  32.558777
9972        NaN        NaN
9973        NaN        NaN
9974        NaN        NaN

[9975 rows x 2 columns]


In [ ]:
def compare_locataion(input_df):
    n = len(input_df)
    results = np.zeros((n,n))

    temp_lat = np.ones((n,n))
    temp_lon = np.ones((n,n))

    lat = np.array(input_df['lat'])
    lon = np.array(input_df['long'])
    temp_lat = temp_lat*lat[:, None]
    temp_lon = temp_lon*lon[:, None]

    temp_lat = np.abs(temp_lat - temp_lat.T)
    temp_lon = np.abs(temp_lon - temp_lon.T)

    temp_lat = np.power(temp_lat , 2)
    temp_lon = np.power(temp_lon , 2)

    results = np.sqrt(temp_lat + temp_lon)
    results = 1 / (results+1) # to handle division by zero

    for i in range(0,n): # set diagonal elements to zero
        results[i][i] = 0    
    results = np.nan_to_num(results) # nans to zero
    #return results
    df = pd.DataFrame(results)
    return df


loc_similarity = compare_locataion(lat_long_df)
print(loc_similarity)

          0         1         2         3     ...      9971  9972  9973  9974
0     0.000000  0.680350  0.270547  0.670282  ...  0.233175   0.0   0.0   0.0
1     0.680350  0.000000  0.248067  0.947369  ...  0.240241   0.0   0.0   0.0
2     0.270547  0.248067  0.000000  0.245087  ...  0.243433   0.0   0.0   0.0
3     0.670282  0.947369  0.245087  0.000000  ...  0.237541   0.0   0.0   0.0
4     0.672327  0.980882  0.247609  0.942705  ...  0.240932   0.0   0.0   0.0
...        ...       ...       ...       ...  ...       ...   ...   ...   ...
9970  0.662959  0.962828  0.246374  0.945649  ...  0.240791   0.0   0.0   0.0
9971  0.233175  0.240241  0.243433  0.237541  ...  0.000000   0.0   0.0   0.0
9972  0.000000  0.000000  0.000000  0.000000  ...  0.000000   0.0   0.0   0.0
9973  0.000000  0.000000  0.000000  0.000000  ...  0.000000   0.0   0.0   0.0
9974  0.000000  0.000000  0.000000  0.000000  ...  0.000000   0.0   0.0   0.0

[9975 rows x 9975 columns]


In [ ]:
# mean similarity dataframe
## and removing recognized similar data

total_mean_similarity = np.mean( np.array([ name_similarity.to_numpy(), phone_similarity.to_numpy(),
                                           category_similarity.to_numpy(), city_similarity.to_numpy(), loc_similarity.to_numpy() ]), axis=0 )

print("Shape of mean the total similatiry matrix is:", len(total_mean_similarity), "x", len(total_mean_similarity[0]))

print("shape of mean similarity:", total_mean_similarity.shape)
over_threshold_idx = np.argwhere(total_mean_similarity > 0.9) # similarity score of 0.9 is the threshold
print("number of simikars found with this threshold:", over_threshold_idx.shape)

prev_line = -1
restlt_df = POI_df.copy()

for i , idx in enumerate(over_threshold_idx):
  if idx[0] != prev_line:
    print("\n \n THIS ROW: \n \n {} , {} , {} , {} , {} \n \n MIGHT HAVE SAME INFO AS ROWS: \n \n {} , {} , {} , {} \n".format(
        str(POI_df.name.loc[[idx[0]]]), POI_df.city.loc[
                                                        [idx[0]]], POI_df.category.loc[[idx[0]]],  POI_df.phones.loc[[idx[0]]], POI_df.location.loc[[idx[0]]] , str(POI_df.name.loc[idx[[1]]]), POI_df.city.loc[idx[[1]]], POI_df.location.loc[[idx[1]]], POI_df.phones.loc[[idx[1]]], POI_df.location.loc[idx[[1]]] ))
    prev_line = idx[0]
  else:
    print("{} , {} , {} , {} , {} \n".format( 
    str(POI_df.name.loc[idx[[1]]]) , POI_df.city.loc[idx[[1]]] , POI_df.location.loc[[idx[1]]], POI_df.phones.loc[[idx[1]]], POI_df.location.loc[idx[[1]]] ))
    
    restlt_df.drop(index = idx[1]  , inplace = True)
    restlt_df.reset_index(drop=True, inplace = True)

print(restlt_df)


Shape of mean the total similatiry matrix is: 9975 x 9975
shape of mean similarity: (9975, 9975)
number of simikars found with this threshold: (78, 2)

 
 THIS ROW: 
 
 106    امداد خودرو کشوری آتیه سازان سبز
Name: name, dtype: object , 106    شهر قزوین
Name: city, dtype: object , 106    تعمیرگاه خودرو
Name: category, dtype: object , 106    [1593]
Name: phones, dtype: object , 106    POINT(49.995249 36.323782)
Name: location, dtype: object 
 
 MIGHT HAVE SAME INFO AS ROWS: 
 
 1656    امداد خودرو آتیه سازان سبز
Name: name, dtype: object , 1656    شهر قزوین
Name: city, dtype: object , 1656    POINT(49.991962 36.31097)
Name: location, dtype: object , 1656    [1593, 09361277038]
Name: phones, dtype: object 


 
 THIS ROW: 
 
 190    بسته بندی اثاث منزل یاسا
Name: name, dtype: object , 190    شهر طهران
Name: city, dtype: object , 190    اداره
Name: category, dtype: object , 190    [02122118714]
Name: phones, dtype: object , 190    POINT(51.4518294 35.8119732)
Name: location, dtype: object 

###Task 2: Category Recommendation

In [ ]:
def nearestNeighbours (commons_df, validation_row): # similarity with respect to validation col/row
    NN_list = []
    for index, row in commons_df.iterrows():
        try:
            val_phrase = list(set(str(validation_row[index]).split()))
        except AttributeError:
            pass
        
        candidate_index = []
        candscores_list = []
        maxlen = max(len(r) for r in row if r is not None)
        maxscore = maxlen/len(val_phrase)
        
        for elind in range(len(row)):
            similarityScore = len(row[elind])/len(val_phrase)
            if similarityScore > 0.7*maxscore and len(row[elind]) > 0: #scores above 70%of maxscore are considered 
                candscores_list.append(similarityScore)
                candidate_index.append(elind)
                
        candidateind_sorted = [x for _, x in 
                               sorted(zip(candscores_list, candidate_index))] # sorted similar candidates with respect to their scores

        NN_list.append(candidateind_sorted)
    return NN_list

In [ ]:
# Using similarity function to give suggestions for missing category value
import re

# selecting rows that have no category
missing_categ_df = POI_df[POI_df['category'].isnull()]
missing_categ_df = missing_categ_df.reset_index(drop=True)
categorized_df = POI_df[POI_df['category'].notnull()]
categorized_df = categorized_df.reset_index(drop=True)
print("number of missing category places:", missing_categ_df.shape[0])
print("number of categorized places:", categorized_df.shape[0])

commonW_df = similarities(missing_categ_df.name, categ_df['دسته بندی']) # based on category names similar to place names
print("common words dataframe shape:", commonW_df.shape)
nn_id = nearestNeighbours(commonW_df, missing_categ_df.name) # a list containing lists of category ids

candidateCateg1 = []
for idlist in nn_id:
  candidateCateg1.append([categ_df['دسته بندی'][x] for x in idlist])


# still we don't have suggestions for all rows
## so we find similarity with categorized data too
commonData_df = similarities(missing_categ_df.name, categorized_df.name) # based on similarity to categorized places
ndata_id = nearestNeighbours(commonData_df, missing_categ_df.name)

# taking frequency of seeing a category in suggestions into account
candidateCateg2 = []
sorted_categs = []
for i, idlist in enumerate(ndata_id):
  candidateCateg2.append([categorized_df.category[x] for x in idlist])

for cand in candidateCateg2:
  sorted_categs.append(sorted(set(cand), key = lambda ele: cand.count(ele), reverse=True))

combined_cands = [a + b for a, b in zip(candidateCateg1, sorted_categs)]

sug_num = [len(x) for x in combined_cands]
print("number of suggestions on each row: {} \n max suggestions for each: {} \n number of zero suggestions: {} out of {} suggestions \n".format(
    sug_num, max(sug_num), sug_num.count(0), len(sug_num)), " which is {} percent of total".format(int(100*sug_num.count(0)/len(sug_num))))


number of missing category places: 645
number of categorized places: 9330
common words dataframe shape: (645, 381)
number of suggestions on each row: [0, 67, 24, 2, 5, 35, 3, 24, 14, 6, 3, 13, 1, 24, 24, 8, 3, 22, 0, 15, 1, 1, 1, 13, 2, 28, 16, 4, 2, 49, 9, 35, 24, 11, 17, 1, 6, 3, 0, 9, 3, 3, 7, 1, 17, 5, 25, 11, 17, 2, 8, 15, 17, 3, 9, 6, 1, 37, 15, 17, 3, 25, 10, 5, 5, 5, 4, 7, 1, 6, 5, 3, 13, 0, 48, 24, 30, 49, 20, 40, 16, 2, 11, 34, 2, 2, 6, 0, 2, 6, 2, 3, 1, 2, 16, 11, 12, 4, 25, 11, 29, 2, 11, 4, 22, 16, 8, 45, 67, 15, 7, 10, 43, 0, 2, 6, 10, 5, 1, 9, 7, 2, 16, 2, 1, 20, 2, 7, 13, 5, 11, 3, 6, 9, 4, 23, 20, 9, 1, 4, 15, 0, 1, 8, 1, 3, 1, 7, 4, 16, 21, 17, 3, 14, 1, 8, 14, 1, 2, 2, 7, 12, 1, 11, 22, 1, 19, 7, 24, 1, 27, 20, 1, 1, 2, 8, 51, 27, 13, 1, 8, 3, 8, 7, 17, 7, 8, 16, 39, 10, 5, 3, 11, 16, 1, 5, 13, 9, 13, 11, 6, 39, 16, 10, 1, 2, 2, 8, 7, 4, 13, 5, 2, 1, 13, 5, 5, 70, 5, 7, 7, 8, 1, 0, 2, 38, 37, 4, 2, 5, 1, 1, 9, 1, 2, 5, 6, 1, 6, 8, 1, 39, 4, 16, 4, 18, 11, 1, 40, 4, 2

In [ ]:
# for the rows remaining with no suggestions 
## we will use phone and location similarity using Task1 functions
zero_sug_indices = np.where(np.array([len(x) for x in combined_cands]) == 0)[0]

zero_sug_df = missing_categ_df.iloc[zero_sug_indices, :] # selects the still-uncategorized rows
zero_sug_loc_sim = loc_similarity.iloc[zero_sug_indices, :]
zero_sug_phone_sim = phone_similarity.iloc[zero_sug_indices, :]

mean_similarity = np.mean( np.array([ zero_sug_phone_sim.to_numpy(), zero_sug_loc_sim.to_numpy() ]), axis=0 )
print("Shape of mean similatiry values matrix is:", len(mean_similarity), "x", len(mean_similarity[0]))

order = mean_similarity.argsort(axis=1)
idx = np.ogrid[tuple(map(slice, mean_similarity.shape))]
idx[1] = order

N_top_valIdx = order[::-1][-10:] # top 10 maximum values

for o in range(len(order.tolist())):
  combined_cands[zero_sug_indices[o]] = POI_df.iloc[order.tolist()[o][-10:], :].category.tolist()

print("\n now we have {} uncategorized rows that have no category suggested!".format([len(x) for x in combined_cands].count(0)))

Shape of mean similatiry values matrix is: 26 x 9975

 now we have 0 uncategorized rows that have no category suggested!


In [ ]:
# EVALUATION phase

from sklearn.model_selection import train_test_split

# splitting the categorized data to subcategorized and evaluation df
# Creating a dataframe with 50% values of original dataframe
subcategorized_df = categorized_df.sample(frac = 0.9) # 90% subcategorized 10% validation
subcategorized_df = subcategorized_df.reset_index(drop=True) 

# Creating dataframe with rest of the 50% values
validation_df = categorized_df.drop(subcategorized_df.index)
validation_df = validation_df.reset_index(drop=True)


commonWords_df = similarities(validation_df.name, categ_df['دسته بندی']) # based on category names similar to uncategorized place names
nearestNeighbour_id = nearestNeighbours(commonWords_df, validation_df.name) # a list containing lists of category ids
candidateCateg1 = []
for idlist in nearestNeighbour_id:
    candidateCateg1.append([categ_df['دسته بندی'][x] for x in idlist])


commonName_df = similarities(validation_df.name, subcategorized_df.name) # based on similarity to subcategorized places
nearestData_id = nearestNeighbours(commonName_df, validation_df.name)
candidateCateg2 = []
sorted_categories = []
for i, idlist in enumerate(nearestData_id):
    candidateCateg2.append([subcategorized_df.category[x] for x in idlist])

for cand in candidateCateg2:
    sorted_categories.append(sorted(set(cand), key = lambda ele: cand.count(ele), reverse=True))



combined_candidates = [a + b for a, b in zip(candidateCateg1, sorted_categories)]
suggestion_num = [len(x) for x in combined_candidates]
print("number of suggestions on each row: {} \n max suggestions for each: {} \n number of zero suggestions: {} out of {} for all data \n".format(
    suggestion_num, max(suggestion_num), suggestion_num.count(0), len(suggestion_num)),
     " which is {} percent of total".format(int(100*suggestion_num.count(0)/len(suggestion_num))))

if suggestion_num.count(0) != 0:
  zero_sug_indices = np.where(np.array([len(x) for x in combined_candidates]) == 0)[0]
  zero_sug_df = validation_df.iloc[zero_sug_indices, :] # selects the still-uncategorized rows
  zero_sug_loc_sim = loc_similarity.iloc[zero_sug_indices, :]
  zero_sug_phone_sim = phone_similarity.iloc[zero_sug_indices, :]
  mean_similarity = np.mean( np.array([ zero_sug_phone_sim.to_numpy(), zero_sug_loc_sim.to_numpy() ]), axis=0 )
  print("Shape of mean similatiry values matrix is:", len(mean_similarity), "x", len(mean_similarity[0]))
  order = mean_similarity.argsort(axis=1)
  idx = np.ogrid[tuple(map(slice, mean_similarity.shape))]
  idx[1] = order
  N_top_valIdx = order[::-1][-10:] # top 10 maximum values
  for o in range(len(order.tolist())):
    combined_candidates[zero_sug_indices[o]] = POI_df.iloc[order.tolist()[o][-10:], :].category.tolist()
  print("\n now we have {} uncategorized rows that have no category suggested!".format([len(x) for x in combined_candidates].count(0)))

number of suggestions on each row: [2, 3, 5, 4, 2, 2, 9, 2, 1, 2, 3, 1, 2, 3, 3, 3, 1, 16, 4, 2, 1, 5, 2, 8, 2, 4, 3, 2, 1, 3, 10, 2, 2, 3, 3, 2, 2, 5, 2, 2, 2, 3, 2, 2, 3, 2, 3, 2, 1, 1, 1, 15, 1, 3, 3, 1, 2, 4, 2, 3, 2, 2, 2, 2, 2, 7, 1, 3, 3, 1, 5, 5, 1, 2, 2, 2, 2, 2, 4, 7, 17, 5, 1, 2, 2, 21, 3, 6, 8, 3, 2, 2, 15, 2, 3, 2, 4, 1, 3, 1, 5, 2, 2, 49, 3, 27, 2, 2, 2, 1, 2, 2, 3, 2, 1, 2, 1, 3, 2, 2, 2, 5, 2, 4, 3, 1, 9, 3, 2, 2, 6, 2, 11, 2, 2, 2, 2, 1, 5, 2, 4, 3, 3, 1, 2, 21, 2, 2, 47, 2, 2, 2, 47, 1, 2, 2, 1, 3, 1, 2, 1, 5, 2, 1, 2, 14, 4, 2, 3, 1, 1, 2, 2, 3, 2, 3, 2, 1, 4, 1, 2, 1, 4, 3, 1, 1, 2, 33, 17, 2, 2, 2, 3, 2, 2, 2, 7, 5, 2, 2, 2, 2, 2, 3, 1, 50, 3, 3, 3, 19, 2, 2, 2, 3, 3, 2, 2, 1, 2, 19, 1, 1, 2, 2, 2, 1, 2, 3, 2, 1, 2, 1, 3, 3, 2, 4, 47, 6, 5, 1, 3, 4, 4, 6, 1, 6, 4, 2, 1, 1, 2, 2, 2, 2, 3, 3, 1, 4, 9, 2, 3, 2, 3, 2, 5, 2, 28, 1, 2, 2, 1, 3, 2, 8, 2, 2, 17, 5, 2, 1, 2, 50, 5, 2, 4, 1, 2, 2, 2, 2, 1, 3, 5, 3, 2, 3, 3, 2, 4, 2, 3, 47, 1, 6, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2,

In [ ]:
# evaluation metric : HIT RATE
hitrate = 0

for i, lst in enumerate(combined_candidates):
  if lst.count(validation_df.category[i]) > 0:
    hitrate += 1

print("There is a correct suugestion for {} percent of validation data".format(round(100*hitrate/len(suggestion_num), 2)))

There is a correct suugestion for 97.75 percent of validation data
